In [ ]:
#!jupyter nbconvert --to=python NPP_timeseries_python3.ipynb

In [ ]:
class NPP_timeseries:
    '''
    class NPP_TotalGlobal(runname,resultpath,savepath,meshpath,first_year,last_year,
                 mapproj='pc',savefig=False,mask="Global Ocean")
                 
    Output:
    self.NPPtotal [Pg C/year]
    self.PhyTotal [Pg C/year]
    self.DiaTotal [Pg C/year]
    self.EPtotal [Pg C/yr]
    self.SiEtotal [Tmol Si/yr]
    
    '''
    def __init__(self,runname,resultpath,savepath,mesh,first_year,last_year,
                 mapproj='pc',
                 plotting = True,
                 savefig=False,
                 mask = "Global Ocean",
                 output = False):

        self.runname = runname
        self.resultpath = resultpath
        self.savepath = savepath
        self.mesh = mesh
        self.fyear = first_year
        self.lyear = last_year
        self.mapproj = mapproj
        self.plotting = plotting
        self.savefig = savefig
        self.mask = mask

        import matplotlib.pyplot as plt
        import numpy as np
        from netCDF4 import Dataset
        import pyfesom2 as pf

        #print('Processing {0}'.format(self.resultpath))

        # load FESOM mesh -------------------------------------------------------------------------------------
        #mesh       = pf.load_mesh(meshpath)
        years = np.arange(self.fyear, self.lyear+1,1)
        
        
        # load nodal area -------------------------------------------------------------------------------------
        meshdiag = pf.get_meshdiag(mesh)
        nod_area_surface = np.ma.masked_equal(meshdiag.nod_area.values, 0)
        
        # get depth of export production
        i_ep_depth = pf.ind_for_depth(100,mesh)
        nod_depth = meshdiag.zbar_n_bottom
        ep_depth = nod_depth[i_ep_depth]
        #print('EP for selected depth = {0} m'.format(ep_depth))

        
        #print('shape nod_area: {0}\nshape nod_area_surface: {1}'.format(np.shape(nod_area),np.shape(nod_area_surface)))
        
        
        # calculating total NPP per year -------------------------------------------------------------------------------
            
        NPPn = pf.get_data(resultpath, "NPPn", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        NPPd = pf.get_data(resultpath, "NPPd", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        DetC = pf.get_data(resultpath, "DetC", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        SiE = pf.get_data(resultpath, "DetSi", years, mesh, how=None, compute=False, runid=self.runname, silent=True)
        ## NPPn:units = "mmolC/(m2*d)"
        
        NPPn = NPPn.resample(time='YS').mean(dim='time')
        NPPd = NPPd.resample(time='YS').mean(dim='time')
        DetC = DetC.resample(time='YS').mean(dim='time')
        SiE = SiE.resample(time='YS').mean(dim='time')
        
        NPP       = 365* (NPPd+NPPn)*12.01 /1e18 # Conversion from [mg/m2/day]   => [mg/m2/yr] => [Pg C/year]
        NPPd       = 365* (NPPd)*12.01 /1e18 
        NPPn       = 365* (NPPn)*12.01 /1e18 
        
        # can provide mask or mask name
        if isinstance(mask, str):
            mask = pf.get_mask(mesh, mask)
        else:
            mask = mask
            
        NPP_timeseries = pf.areasum_data(NPP,mesh,mask)
        NPPd_timeseries = pf.areasum_data(NPPd,mesh,mask)
        NPPn_timeseries = pf.areasum_data(NPPn,mesh,mask)
        
        del NPP, NPPd, NPPn
        
        ## Carbon export, DetC:units = "[mmol/m3]"
        Vdet = 0.0288 * 100. + 20. ## sinking velocity
        DetC = 365. * DetC * 12.01 * Vdet /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
        detct=DetC[:,:,i_ep_depth]
        EP_timeseries = pf.areasum_data(detct,mesh,mask)
        
        del DetC, detct
        
        ## Si export, DetSi:units = "[mmol/m3]"
        #SiE = 365. * SiE *28.085 * Vdet /1e18 # [mmol/m3] => [mg/m2/yr] => [Pg C/yr]
        SiE = 365. * SiE * Vdet /1e15 # [mmol/m3] => [mmol/m2/yr]  =>  [Tmol Si/yr]      
        siect = SiE[:,:,i_ep_depth]
        SiE_timeseries = pf.areasum_data(siect,mesh,mask)   
        
        del SiE, siect
        
        if output:
            self.NPPtotal = NPP_timeseries
            self.PhyTotal = NPPn_timeseries
            self.DiaTotal = NPPd_timeseries
            self.EPtotal = EP_timeseries
            self.SiEtotal = SiE_timeseries
        
        if plotting:
            # plotting total NPP -------------------------------------------------------------------------------        
            fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
            plt.subplot(2, 3, 1)
            plt.plot(years,NPP_timeseries,'.-',color='C0',label='Total')
            plt.title('Total NPP')
            plt.ylabel(r'[Pg C yr$^{-1}$]')

            plt.subplot(2, 3, 2)
            plt.plot(years,NPPn_timeseries,'.-g',label='Nano',color='C1')
            plt.title('nanophytoplankton NPP')

            plt.subplot(2, 3, 3)
            plt.plot(years,NPPd_timeseries,'.-r',label='Dia',color='C2')
            plt.title(' diatoms NPP')

            # all NPP together
            plt.subplot(2, 3, 4)
            plt.plot(years,NPP_timeseries,'.-',label='Total',color='C0')
            plt.plot(years,NPPn_timeseries,'.-',label='Nano',color='C1')
            plt.plot(years,NPPd_timeseries,'.-',label='Dia',color='C2')
            plt.title('NPP')
            plt.ylabel(r'[Pg C yr$^{-1}$]')
            plt.legend(loc='best')

            # Export production
            plt.subplot(2, 3, 5)
            plt.plot(years,EP_timeseries,'.-',color='C3')
            plt.title('EP at 100 m')
            plt.ylabel(r'[Pg C yr$^{-1}$]')

            # Si export
            plt.subplot(2, 3, 6)
            plt.plot(years,SiE_timeseries,'.-',color='C4')
            plt.title('Si export at 100 m')
            plt.ylabel(r'[Tmol Si yr$^{-1}$]')


            if(savefig):
                plt.savefig(self.savepath+self.runname+'_'+'NPP_TotalGlobal_'+str(self.fyear)+'to'+str(self.lyear)+'.png', dpi = 300, bbox_inches='tight')
            plt.show(block=False)
        
    